# Tests

In [1]:
Threads.nthreads()

1

In [2]:
using LinearAlgebra
using Turing
using LimberJack
using NPZ
using ForwardDiff
using BenchmarkTools;
using Interpolations

In [3]:
using PythonCall
np = pyimport("numpy");

    CondaPkg Found dependencies: /home/jaimerz/.julia/environments/v1.7/CondaPkg.toml
    CondaPkg Resolving changes
             - libstdcxx-ng
             - python
    CondaPkg Removing packages
             │ /home/jaimerz/.julia/artifacts/134b3f7254989e369343bab626bc7e0c77a9cc1a/bin/micromamba
             │ -r /home/jaimerz/.julia/scratchspaces/0b3b1443-0f03-428d-bdfb-f27f9c1191ea/root
             │ remove
             │ -y
             │ -p /home/jaimerz/.julia/environments/v1.7/.CondaPkg/env
             │ libstdcxx-ng
             └ python
    CondaPkg Installing packages
             │ /home/jaimerz/.julia/artifacts/134b3f7254989e369343bab626bc7e0c77a9cc1a/bin/micromamba
             │ -r /home/jaimerz/.julia/scratchspaces/0b3b1443-0f03-428d-bdfb-f27f9c1191ea/root
             │ install
             │ -y
             │ -p /home/jaimerz/.julia/environments/v1.7/.CondaPkg/env
             │ --override-channels
             │ --no-channel-priority
             │ numpy[version='


                                           __
          __  ______ ___  ____ _____ ___  / /_  ____ _
         / / / / __ `__ \/ __ `/ __ `__ \/ __ \/ __ `/
        / /_/ / / / / / / /_/ / / / / / / /_/ / /_/ /
       / .___/_/ /_/ /_/\__,_/_/ /_/ /_/_.___/\__,_/
      /_/

Transaction

  Prefix: /home/jaimerz/.julia/environments/v1.7/.CondaPkg/env

  Removing specs:

   - libstdcxx-ng
   - python


  Package               Version  Build                Channel           Size
──────────────────────────────────────────────────────────────────────────────
  Remove:
──────────────────────────────────────────────────────────────────────────────

  - _libgcc_mutex           0.1  conda_forge          conda-forge     Cached
  - _openmp_mutex           4.5  2_gnu                conda-forge     Cached
  - bzip2                 1.0.8  h7f98852_4           conda-forge     Cached
  - ca-certificates   2022.12.7  ha878542_0           conda-forge     Cached
  - ld_impl_linux-64       2.39  hcc3a1bd_1

In [4]:
meta = np.load("../../data/ND/ND_meta.npz")
names = pyconvert(Vector{String}, meta["names"])
types = pyconvert(Vector{String}, meta["types"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"])
idx = pyconvert(Vector{Int}, meta["idx"])
data_vector = pyconvert(Vector{Float64}, meta["cls"])
cov_tot = pyconvert(Matrix{Float64}, meta["cov"])
inv_cov_tot = pyconvert(Matrix{Float64}, meta["inv_cov"]);

In [5]:
print(length(data_vector))

665

In [6]:
files = npzread("../../data/ND/ND_files.npz");

In [7]:
path = "/home/jaimerz/PhD/LimberJack.jl/chains/carlos_chains/cl_cross_corr_v3_DES_K1000_all_mag_correctMag/"
pars = np.loadtxt(string(path, "cl_cross_corr_v3_DES_K1000_all_mag_correctMag.bestfit"))
pars = pyconvert(Vector{Float64}, pars);


In [8]:
cls_carlos_ND = np.load(string(path, "cl_cross_corr_bestfit_info.npz"))
cls_carlos_SD = np.load(string(path, "cl_cross_corr_bestfit_info_copy.npz"))
cls_carlos_FD = np.append(cls_carlos_SD["cls"], cls_carlos_ND["cls"])
cls_carlos_FD = pyconvert(Vector{Float64}, cls_carlos_FD)
carlos_chi = pyconvert(Float64, - (cls_carlos_ND["chi2"] +  cls_carlos_SD["chi2"]));

## Timings

In [9]:
function diff_Xi2(pars; data_vector=data_vector, inv_cov=inv_cov_tot)
    nuisances = Dict("DESgc__0_0_b" => pars[6],
                     "DESgc__1_0_b" => pars[7],
                     "DESgc__2_0_b" => pars[8],
                     "DESgc__3_0_b" => pars[9],
                     "DESgc__4_0_b" => pars[10],
                     "DESgc__0_0_dz" => pars[11],
                     "DESgc__1_0_dz" => pars[12],
                     "DESgc__2_0_dz" => pars[13],
                     "DESgc__3_0_dz" => pars[14],
                     "DESgc__4_0_dz" => pars[15],
        
                     "A_IA" => pars[16],
                     "alpha_IA" => pars[17],
        
                     "DESwl__0_e_m" => pars[18],
                     "DESwl__1_e_m" => pars[19],
                     "DESwl__2_e_m" => pars[20],
                     "DESwl__3_e_m" => pars[21],
                     "DESwl__0_e_dz" => pars[22],
                     "DESwl__1_e_dz" => pars[23],
                     "DESwl__2_e_dz" => pars[24],
                     "DESwl__3_e_dz" => pars[25],
        
                     "eBOSS__0_0_b" => pars[26],
                     "eBOSS__1_0_b" => pars[27],

                     "DECALS__0_0_b" => pars[28],
                     "DECALS__1_0_b" => pars[29],
                     "DECALS__2_0_b" => pars[30],
                     "DECALS__3_0_b" => pars[31],
                     "DECALS__0_0_dz" => pars[32],
                     "DECALS__1_0_dz" => pars[33],
                     "DECALS__2_0_dz" => pars[34],
                     "DECALS__3_0_dz" => pars[35],
                    
                     "KiDS1000__0_e_m" => pars[36],
                     "KiDS1000__1_e_m" => pars[37],
                     "KiDS1000__2_e_m" => pars[38],
                     "KiDS1000__3_e_m" => pars[39],
                     "KiDS1000__4_e_m" => pars[40],
                     "KiDS1000__0_e_dz" => pars[41],
                     "KiDS1000__1_e_dz" => pars[42],
                     "KiDS1000__2_e_dz" => pars[43],
                     "KiDS1000__3_e_dz" => pars[44],
                     "KiDS1000__4_e_dz" => pars[45])
    
    Ωm = pars[1] + pars[2]
    s8 = pars[46]
    Ωb = pars[1]
    h = pars[5]
    ns = pars[4]
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8, 
                                     tk_mode="emulator",
                                     Pk_mode="Halofit", 
                                     emul_path= "../../emulator/files.npz")
    
    theory = Theory(cosmology, names, types, pairs,
                    idx, files; Nuisances=nuisances)
    diff = @.(theory-data_vector)
    return dot(diff, inv_cov_tot  * diff)

end

diff_Xi2 (generic function with 1 method)

In [10]:
function fd(pars)
    n = length(pars)
    grad  = zeros(length(pars))
    for i in 1:n
        d = zeros(n)
        e = 0.005
        d[i] +=  e
        grad[i] = (diff_Xi2(pars .+ d) .- diff_Xi2(pars .- d)) ./ (2*e)
    end
    return grad
end

fd (generic function with 1 method)

In [11]:
@benchmark fd(pars)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 14.071 s (5.44% GC) to evaluate,
 with a memory estimate of 9.43 GiB, over 34348666 allocations.

In [ ]:
@benchmark ForwardDiff.gradient(diff_Xi2, pars)

In [12]:
@benchmark diff_Xi2(pars)

BenchmarkTools.Trial: 176 samples with 1 evaluation.
 Range (min … max):  19.948 ms … 51.473 ms  ┊ GC (min … max): 0.00% … 26.49%
 Time  (median):     24.011 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   28.390 ms ±  6.781 ms  ┊ GC (mean ± σ):  6.83% ± 11.92%

        █▂                                                     
  ▂▂▁▄▄▆███▄▄▃▁▁▂▂▁▁▁▁▃▃▃▄▃▂▄▃▂▄▂▃▄▄▃▄▄▅▃▁▃▁▃▁▁▂▂▁▁▁▂▂▁▂▁▁▁▂▂ ▂
  19.9 ms         Histogram: frequency by time        46.5 ms <

 Memory estimate: 43.50 MiB, allocs estimate: 155416.

In [ ]:
npzwrite(string(, ".npz"), [evall, diff])

In [16]:
using ReverseDiff
ftape_diff_Xi2 = ReverseDiff.GradientTape(diff_Xi2, pars)
compiled_ftape_diff_Xi2 = ReverseDiff.compile(ftape_diff_Xi2)

typename(ReverseDiff.CompiledTape)(diff_Xi2)

In [17]:
Xi2 = similar(pars);

In [18]:
@benchmark ReverseDiff.gradient!(Xi2, compiled_ftape_diff_Xi2, pars)

LoadError: InterruptException:

In [15]:
@benchmark ForwardDiff.jacobian(diff_cls, pars)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  2.305 s …   2.404 s  ┊ GC (min … max): 8.40% … 8.07%
 Time  (median):     2.340 s              ┊ GC (median):    8.29%
 Time  (mean ± σ):   2.350 s ± 50.542 ms  ┊ GC (mean ± σ):  8.35% ± 0.26%

  █                   █                                   █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.3 s          Histogram: frequency by time         2.4 s <

 Memory estimate: 3.76 GiB, allocs estimate: 1122396.

In [ ]:
@benchmark ReverseDiff.jacobian(diff_cls, pars)

## Xi2

In [43]:
best_pred = make_cls(pars)

610-element Vector{Float64}:
 3.729976949631498e-5
 1.924863264546964e-5
 1.2198839863768642e-5
 2.7338998158421804e-5
 1.8302492387509104e-5
 1.1697676470225297e-5
 8.461577277803604e-6
 6.2847518106000625e-6
 4.953759872071052e-6
 4.0277958603799074e-6
 2.0247593758184622e-5
 1.3879505201281076e-5
 1.0208564507478019e-5
 ⋮
 1.481014723350936e-9
 1.2597256449462859e-9
 1.073704667701235e-9
 9.174815729181471e-10
 7.858916381457959e-10
 6.74149817464629e-10
 5.784141317368861e-10
 4.954936101687788e-10
 4.234671163674685e-10
 3.6091137647921583e-10
 3.0662205257958187e-10
 2.5945231667605537e-10

In [44]:
best_Xi = dot((best_pred .- data_vector), inv_cov_tot  * (best_pred .- data_vector))

726.9778969921368

In [45]:
reduced_best_Xi = round(best_Xi/(length(data_vector)-43), digits=3)

1.282

## Prior sampling

In [21]:
N_samples = 100
cls_preds = zeros(N_samples, length(data_vector))
for i in 1:N_samples
    Ωm = rand(Uniform(0.1, 0.6))
    s8 = rand(Uniform(0.6, 0.9))
    cls_preds[i,:] = make_cls(pars; Ωm=Ωm, s8=s8)
end

In [22]:
cls_preds_m, cls_preds_s = mean(cls_preds, dims=1), std(cls_preds, dims=1);

In [23]:
npzwrite("jaime_cls.npz", Dict("cls_best" => best_pred,
                               "cls_m" => cls_preds_m,
                               "cls_s" => cls_preds_s))

## Turing

In [9]:
@model function model(data_vector::Vector{Float64})
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = 0.05 #~ Uniform(0.03, 0.07)
    h = 0.67 #~ Uniform(0.6, 0.9)
    s8 = 0.81 #~ Uniform(0.6, 1.0)
    ns = 0.96 #~ Uniform(0.87, 1.07)

    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, s8, s8,
                                     tk_mode="emulator",
                                     Pk_mode="Halofit")
    theory = Theory(cosmology,
                    tracers_names, pairs,
                    idx, files)
    data_vector ~ MvNormal(theory, cov_tot)
    return data_vector
end;

In [10]:
iterations = 300
nadapts = 100;

In [11]:
chain3 = sample(model(data_vector), NUTS(5, 0.1; init_ϵ=0.03), iterations,
                progress=true; save_state=true);

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
Sampling:   1%|▍                                        |  ETA: 0:07:02┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
Sampling: 100%|█████████████████████████████████████████| Time: 0:01:28


LoadError: InterruptException:

In [12]:
chain1 = sample(model(data_vector), MH(), iterations,
            nadapt=nadapts, progress=true; save_state=true);

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:24


LoadError: InterruptException:

In [4]:
files = npzread("../data/FD/K1K_DELS_DESY1_eBOSS_CMBk_files.npz")
products = npzread("products.npz")

Dict{String, Array} with 5 entries:
  "idx"         => [0, 5, 13, 23, 34, 47, 52, 57, 62, 67  …  1077, 1099, 1121, …
  "cls_m"       => [0.000229969 7.55142e-5 … 3.71104e-10 3.14893e-10]
  "err"         => [5.919e-5, 1.57008e-5, 6.64192e-6, 3.82552e-6, 2.56294e-6, 2…
  "cls_s"       => [0.000216359 3.16836e-5 … 2.44088e-10 2.0759e-10]
  "data_vector" => [0.000190242, 7.39128e-5, 4.26125e-5, 2.22966e-5, 2.17688e-5…

In [3]:
figure, axis = plt.subplots(5, figsize=(5,14))
l = 1
for i in 0:4
    axis[i+1].errorbar(files[string("ls_1", i, "1", i)], 
                     data[idx[l]:idx[l+1]-1], 
                     yerr=err[idx[l]:idx[l+1]-1], fmt="gs", label="Data")
    axis[i+1].plot(files[string("ls_1", i, "1", i)],
                   cls_preds_m[idx[l]:idx[l+1]-1],
                   "bo-", label="Preds")
    axis[i+1].fill_between(files[string("ls_1", i, "1", i)],
                   cls_preds_m[idx[l]:idx[l+1]-1].+cls_preds_s[idx[l]:idx[l+1]-1],
                   cls_preds_m[idx[l]:idx[l+1]-1].-cls_preds_s[idx[l]:idx[l+1]-1],
                   color="b", alpha=.5)
    axis[i+1].set_title(string("GC", i, "GC", i))
    l += 1
end
plt.legend()
plt.show()
print(l)

LoadError: UndefVarError: files not defined